In [ ]:
import os
import requests
from dotenv import load_dotenv
import pandas as pd

In [25]:
load_dotenv(".env")
api_key = os.getenv("API_KEY")
if not api_key:
    raise ValueError("API_KEY not found.") 
movie_ids = [
    0, 299534, 19995, 140607, 299536, 597, 135397, 420818,
    24428, 168259, 99861, 284054, 12445, 181808, 330457,
    351286, 109445, 321612, 260513
]
selected_movie_ids = []
for movie_id in movie_ids:
    if movie_id == 0:
        continue
    url = f"https://api.themoviedb.org/3/movie/{movie_id}?append_to_response=credits&api_key={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        selected_movie_ids.append(response.json())
    else:
        print(f"Failed to fetch movie {movie_id}: {response.status_code}")

In [26]:
df_movie = pd.DataFrame(selected_movie_ids) #Converted DataFrame

In [27]:
df_movie['id']

0     299534
1      19995
2     140607
3     299536
4        597
5     135397
6     420818
7      24428
8     168259
9      99861
10    284054
11     12445
12    181808
13    330457
14    351286
15    109445
16    321612
17    260513
Name: id, dtype: int64

In [28]:
df_movie.columns

Index(['adult', 'backdrop_path', 'belongs_to_collection', 'budget', 'genres',
       'homepage', 'id', 'imdb_id', 'origin_country', 'original_language',
       'original_title', 'overview', 'popularity', 'poster_path',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'video', 'vote_average', 'vote_count', 'credits'],
      dtype='object')

In [29]:
# Step 2: Data Cleaning and Preprocessing
# the irrelevant columns to drop
cols_to_drop = ['adult', 'imdb_id', 'original_title', 'video', 'homepage']
for col in cols_to_drop:
    # Check if the column actually exists in the DataFrame to avoid errors
    if col in cols_to_drop:
        # axis=1 refers to columns. inplace=True modifies the DataFrame directly.
        df_movie.drop(col, axis=1, inplace=True)



In [30]:
df_movie.columns

Index(['backdrop_path', 'belongs_to_collection', 'budget', 'genres', 'id',
       'origin_country', 'original_language', 'overview', 'popularity',
       'poster_path', 'production_companies', 'production_countries',
       'release_date', 'revenue', 'runtime', 'spoken_languages', 'status',
       'tagline', 'title', 'vote_average', 'vote_count', 'credits'],
      dtype='object')